<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-01 14:16:31
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.36 C
-------------------
Today PnL: -67.80 K (-0.5%)
Current PnL: -18.61 L (-12.66%)
CY Booked + Current PnL: -10.34 L (-7.03%)
-------------------
Total profit:  2.44 L
Total loss:  -21.05 L
-------------------
Total Booked + Current PnL: 16.64 L (13.86%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.07%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 76.94 L (56.46%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-21.55,H-LC,5.25,108328.0,4016.0,13519.0,-0.88,3.85,12.48,16.81,10.0,0.30,0.82,20.51,X5K,ATH,METALS
71,SIS,477.00,477.00,2327.46,H-SC,6.79,65314.0,-15776.0,15773.0,-0.57,-19.45,24.15,0.00,157.0,-1.00,0.49,32.37,OX40N,NTT,MISC
27,DIXON,14367.00,18940.15,-26.34,H-MC,6.12,135560.0,20624.0,15955.0,0.62,17.94,11.77,31.83,85.0,1.29,1.02,52.56,X40N,ATH,IT
55,MEDANTA,1087.93,1486.00,20.82,H-SC,12.53,154825.0,28625.0,17557.0,1.79,22.68,11.34,36.59,156.0,1.63,1.17,38.04,XY24,NTT,HEALTHCARE
77,TATAELXSI,7332.28,7332.00,-19.16,H-MC,1.91,84945.0,-17707.0,17703.0,-0.43,-17.25,20.84,-0.00,101.0,-1.00,0.64,29.13,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,2347.98,4546.37,7.94,H-SC,19.89,100405.0,-558.0,95094.0,-0.56,-0.55,94.71,93.63,149.0,-0.01,0.76,49.55,SR,ATH,CHEMICALS
19,CAMS,3643.00,5226.82,-8.59,H-SC,0.82,102620.0,616.0,43726.0,-1.92,0.60,42.61,43.48,120.0,0.01,0.77,17.48,X40N,ATH,MISC
41,ICICIPRULI,600.83,790.00,-15.89,H-MC,1.33,135781.0,-7.0,42757.0,-2.46,-0.00,31.49,31.48,111.0,-0.00,1.02,12.11,X40,ATH,INSURANCE
35,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,3.66,150682.0,-231.0,57530.0,1.04,-0.15,38.18,37.97,90.0,-0.00,1.14,25.11,AR,ATH,AUTO
91,WIPRO,243.46,420.00,-11.94,M-LC,5.19,150685.0,-260.0,109714.0,-2.12,-0.17,72.81,72.51,55.0,-0.00,1.14,5.84,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-21.82,H-LC,1.71,206048.0,1548.0,39870.0,-3.25,0.76,19.35,20.25,9.0,0.04,1.55,9.56,X40N,BTT,PHARMA
53,LTIM,5564.16,7230.2,-5.37,H-LC,2.82,181926.0,-18384.0,78356.0,-1.03,-9.18,43.07,29.94,24.0,-0.23,1.37,25.68,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.62,H-LC,2.88,185369.0,-15137.0,63915.0,-0.74,-7.55,34.48,24.33,30.0,-0.24,1.40,6.89,X40N,BTT,FINANCE
5,ASIANPAINT,2961.56,4250.0,-16.76,H-LC,3.13,165213.0,-36173.0,123794.0,1.40,-17.96,74.93,43.51,19.0,-0.29,1.24,15.00,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-15.77,H-LC,4.51,267146.0,1720.0,55192.0,1.58,0.65,20.66,21.44,26.0,0.03,2.01,7.33,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,BSOFT,628.85,836.99,-16.31,H-SC,12.67,70262.0,-39787.0,76213.0,2.55,-36.15,108.47,33.10,129.0,-0.52,0.53,16.75,XR,ATH,IT
50,KPIGREEN,497.21,731.64,29.10,H-SC,12.86,132136.0,6839.0,52233.0,2.26,5.46,39.53,47.15,144.0,0.13,1.00,65.31,MH,ATH,POWER
49,KANSAINER,299.63,340.00,-67.65,H-SC,1.68,224730.0,-44937.0,81262.0,1.96,-16.66,36.16,13.47,136.0,-0.55,1.69,13.67,XY24,NTT,PAINTS
68,SFL,1016.35,1287.00,31.97,M-SC,13.20,174189.0,-88029.0,157850.0,1.95,-33.57,90.62,26.63,231.0,-0.56,1.31,8.30,XY24,NTT,MISC
55,MEDANTA,1087.93,1486.00,20.82,H-SC,12.53,154825.0,28625.0,17557.0,1.79,22.68,11.34,36.59,156.0,1.63,1.17,38.04,XY24,NTT,HEALTHCARE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-307.74,H-SC,20.41,88400.0,-44886.0,129479.0,-3.86,-33.68,146.47,63.47,147.0,-0.35,0.67,48.18,XR,NTT,CHEMICALS
21,CIPLA,1492.70,1795.00,-21.82,H-LC,1.71,206048.0,1548.0,39870.0,-3.25,0.76,19.35,20.25,9.0,0.04,1.55,9.56,X40N,BTT,PHARMA
60,RAJESHEXPO,517.65,518.00,1675.25,L-SC,3.64,48423.0,-88754.0,88847.0,-3.05,-64.70,183.48,0.07,266.0,-1.00,0.36,20.43,OX40N,NTT,JEWELLERY
40,ICICIGI,1839.64,2260.25,-15.47,H-LC,5.05,138484.0,2351.0,28777.0,-2.89,1.73,20.78,22.86,21.0,0.08,1.04,14.56,X40,ATH,INSURANCE
41,ICICIPRULI,600.83,790.00,-15.89,H-MC,1.33,135781.0,-7.0,42757.0,-2.46,-0.00,31.49,31.48,111.0,-0.00,1.02,12.11,X40,ATH,INSURANCE


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,5.79,148800.0,-25759.0,25787.0,0.00,-14.76,17.33,0.02,135.0,-1.00,1.12,29.03,OX40N,NTT,PAINTS
75,SYMPHONY,1306.42,1306.0,-32.98,M-SC,8.90,142790.0,-28351.0,28301.0,-2.23,-16.57,19.82,-0.03,197.0,-1.00,1.08,3.33,OX40N,NTT,DURABLES
49,KANSAINER,299.63,340.0,-67.65,H-SC,1.68,224730.0,-44937.0,81262.0,1.96,-16.66,36.16,13.47,136.0,-0.55,1.69,13.67,XY24,NTT,PAINTS
77,TATAELXSI,7332.28,7332.0,-19.16,H-MC,1.91,84945.0,-17707.0,17703.0,-0.43,-17.25,20.84,-0.00,101.0,-1.00,0.64,29.13,OX40N,NTT,IT
83,UJJIVANSFB,52.77,53.0,35.36,H-SC,20.81,117504.0,-24975.0,25592.0,-1.69,-17.53,21.78,0.44,163.0,-0.98,0.89,38.69,OX40N,NTT,BANKS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-28.02,H-LC,13.86,234850.0,-61084.0,155001.0,-0.85,-20.64,66.00,31.73,1.0,-0.39,1.77,0.00,X40,BTT,IT
88,VBL,492.64,671.64,-8.97,H-LC,11.75,276263.0,11223.0,85089.0,-1.65,4.23,30.80,36.33,3.0,0.13,2.08,18.10,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-17.80,H-LC,7.45,264025.0,2424.0,143207.0,-2.25,0.93,54.24,55.67,4.0,0.02,1.99,7.09,X40,BTT,IT
1,ABB,5551.76,7934.00,-40.25,H-LC,7.60,213135.0,-3384.0,96294.0,-0.82,-1.56,45.18,42.91,5.0,-0.04,1.61,11.41,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.09,H-LC,7.46,177632.0,-13014.0,59524.0,-0.81,-6.83,33.51,24.40,8.0,-0.22,1.34,8.02,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,20.82,H-SC,12.53,154825.0,28625.0,17557.0,1.79,22.68,11.34,36.59,156.0,1.63,1.17,38.04,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.10,H-SC,12.86,132136.0,6839.0,52233.0,2.26,5.46,39.53,47.15,144.0,0.13,1.00,65.31,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,20.82,H-SC,12.53,154825.0,28625.0,17557.0,1.79,22.68,11.34,36.59,156.0,1.63,1.17,38.04,XY24,NTT,HEALTHCARE
25,DABUR,505.20,735.00,-3.23,H-MC,5.16,209722.0,12694.0,76926.0,1.65,6.44,36.68,45.49,100.0,0.17,1.58,20.32,XY24,BTT,FMCG
19,CAMS,3643.00,5226.82,-8.59,H-SC,0.82,102620.0,616.0,43726.0,-1.92,0.60,42.61,43.48,120.0,0.01,0.77,17.48,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-8.95,H-SC,9.51,170750.0,6080.0,61880.0,-1.76,3.69,36.24,41.28,119.0,0.10,1.29,11.97,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.10,H-SC,12.86,132136.0,6839.0,52233.0,2.26,5.46,39.53,47.15,144.0,0.13,1.00,65.31,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-47.33,H-SC,13.36,137291.0,13955.0,119814.0,0.05,11.31,87.27,108.45,118.0,0.12,1.03,36.20,AR,ATH,MISC
90,WHIRLPOOL,1219.98,2270.00,-40.58,M-SC,7.03,100072.0,8574.0,70180.0,-0.02,9.37,70.13,86.07,211.0,0.12,0.75,44.77,XR,NTT,DURABLES
91,WIPRO,243.46,420.00,-11.94,M-LC,5.19,150685.0,-260.0,109714.0,-2.12,-0.17,72.81,72.51,55.0,-0.00,1.14,5.84,XR,NTT,IT
35,HEROMOTOCO,4311.81,5949.07,-6.19,H-MC,3.66,150682.0,-231.0,57530.0,1.04,-0.15,38.18,37.97,90.0,-0.00,1.14,25.11,AR,ATH,AUTO


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2327.46,H-SC,6.79,65314.0,-15776.0,15773.0,-0.57,-19.45,24.15,0.00,157.0,-1.00,0.49,32.37,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.16,68544.0,-45005.0,85056.0,-0.14,-39.63,124.09,35.27,267.0,-0.53,0.52,67.76,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-19.60,H-SC,16.74,91620.0,-6360.0,28228.0,-0.86,-6.49,30.81,22.32,151.0,-0.23,0.69,32.88,XR,ATH,FINANCE
3,ALKYLAMINE,2347.98,4546.37,7.94,H-SC,19.89,100405.0,-558.0,95094.0,-0.56,-0.55,94.71,93.63,149.0,-0.01,0.76,49.55,SR,ATH,CHEMICALS
83,UJJIVANSFB,52.77,53.00,35.36,H-SC,20.81,117504.0,-24975.0,25592.0,-1.69,-17.53,21.78,0.44,163.0,-0.98,0.89,38.69,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,20.82,H-SC,12.53,154825.0,28625.0,17557.0,1.79,22.68,11.34,36.59,156.0,1.63,1.17,38.04,XY24,NTT,HEALTHCARE
27,DIXON,14367.00,18940.15,-26.34,H-MC,6.12,135560.0,20624.0,15955.0,0.62,17.94,11.77,31.83,85.0,1.29,1.02,52.56,X40N,ATH,IT
51,LALPATHLAB,2838.53,3545.00,-23.82,H-SC,5.55,216702.0,23682.0,24357.0,1.14,12.27,11.24,24.89,128.0,0.97,1.63,38.22,X40N,NTT,HEALTHCARE
88,VBL,492.64,671.64,-8.97,H-LC,11.75,276263.0,11223.0,85089.0,-1.65,4.23,30.80,36.33,3.0,0.13,2.08,18.10,X40N,ATH,FMCG
69,SHALBY,261.39,327.00,962.20,M-SC,8.99,144490.0,-36653.0,82114.0,-0.54,-20.23,56.83,25.10,233.0,-0.45,1.09,15.79,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.71
1,25,41.65
2,50,71.31


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.83
LC    37.11
MC    22.06
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.04
X40N     15.14
X40      13.17
XY25     11.77
XR        8.96
OX40N     7.77
AR        7.66
X5K       2.19
MH        1.73
X200      1.37
SR        1.20
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.07
H-SC    27.32
H-MC    19.16
M-SC    12.08
M-LC     6.99
M-MC     2.56
L-SC     1.43
L-LC     1.05
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.44,-0.53,32.99
IT,11.95,-17.68,74.84
FINANCE,8.36,-17.62,60.30
BANKS,7.40,-16.12,63.49
PAINTS,5.79,-13.43,35.79
MISC,5.14,-19.42,89.29
HEALTHCARE,5.11,3.20,21.24
ELECTRICAL,4.82,-10.02,57.67
AUTO,4.29,-18.56,70.14


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2781802.0,23
XR,1103825.0,13
AR,1048238.0,9
X40,823510.0,10
X40N,597894.0,12
XY25,492256.0,8
OX40N,346605.0,10
SR,184653.0,2
X5K,149507.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2698138.0,30
M-SC,1365706.0,15
H-LC,1298837.0,20
H-MC,1163188.0,15
M-LC,451366.0,5
M-MC,337468.0,2
L-SC,270958.0,3
L-MC,57376.0,1
L-LC,50692.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,928445.0,7
M-SC,XY24,839067.0,7
H-SC,AR,612471.0,4
H-LC,X40,550481.0,6
H-SC,XR,515988.0,6
H-MC,XY24,449416.0,4
M-MC,XY24,337468.0,2
H-LC,X40N,300333.0,6
H-SC,X40N,222768.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
